# Обкачка страниц

In [20]:
import requests
from bs4 import BeautifulSoup
from operator import attrgetter, itemgetter
from tqdm.notebook import tqdm
import itertools
import pandas as pd
import re
import time
import sys

In [21]:
ROOT = "https://shop.relod.ru"
CAT_PATH = "https://shop.relod.ru/catalog-products/4577"

Вспомогательные функции:

In [22]:
def get_page(url, params=None, n_attempts=10, t_sleep=2):
    assert n_attempts >= 1 and t_sleep > 0
    for i in range(n_attempts):
        page = requests.get(url, params=params)
        if page.ok:
            break
        
        time.sleep(t_sleep)
    if not page.ok:
        return (page.ok, page.status_code)
    return (page.ok, page)

In [23]:
def get_links_from_html(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    tags = soup.find_all(name='a', id=lambda s : s and s.startswith('bx_'))
    if len(tags) == 0:
        print("No tags a with required parameters", file=sys.stderr)
        return []
    links = ["".join((ROOT, tag['href'])) for tag in tags]
    return links

## Реализуем простой способ обкачки

In [24]:
def get_links_from_page(page_num):
    params = {
        "sort" : "PROPERTY_RATING",
        "order" : "desc",
        "PAGEN_1" : page_num
    }
    ok, page = get_page(CAT_PATH, params)
    if not ok:
        print("Page with base {} and params {} couldn't "
              "be loaded. Status code: {}".format(CAT_PATH, params, page), file=sys.stderr)
        return []
    links = get_links_from_html(page.text)
    return links

In [25]:
def get_all_links(start_page_num=1, finish_page_num=250):
    res = [get_links_from_page(page_num) for page_num in tqdm(range(start_page_num, finish_page_num + 1))]
    links = list(itertools.chain(*res))
    return links

In [26]:
links = get_all_links()
len(list(links))

5000

## Реализуем способ обкачки посложнее

Параметры пост-запроса для "Показать еще":

In [28]:
ajax_url = 'https://shop.relod.ru/bitrix/components/bxready.market2/catalog.section/ajax.php'
headers = {'referer' : 'https://shop.relod.ru/catalog-products/4577/?sort=PROPERTY_RATING&order=desc',
          'accept-encoding' : 'gzip, deflate, br'}
data = {
    'siteId' : 's1',
    'template' : '.default.5066805247fd90f4681c9458759019c47aeeee73c2035d99874d41cd9e7eb382',
    'parameters' : 'YToxNzM6e3M6MTg6IkNPTVBPTkVOVF9URU1QTEFURSI7czo4OiIuZGVmYXVsdCI7czoxMToiSUJMT0NLX1RZUEUiO3M6NzoiY2F0YWxvZyI7czo5OiJJQkxPQ0tfSUQiO3M6MToiMiI7czoxMDoiU0VDVElPTl9JRCI7czo0OiI0NTc3IjtzOjEyOiJTRUNUSU9OX0NPREUiO047czoxMToiRklMVEVSX05BTUUiO3M6MTY6IkNBVEFMT0dfUFJPRFVDVFMiO3M6MTk6IklOQ0xVREVfU1VCU0VDVElPTlMiO3M6MToiQSI7czoxOToiU0hPV19BTExfV09fU0VDVElPTiI7czoxOiJOIjtzOjEzOiJDVVNUT01fRklMVEVSIjtzOjA6IiI7czoxODoiSElERV9OT1RfQVZBSUxBQkxFIjtzOjE6Ik4iO3M6MjU6IkhJREVfTk9UX0FWQUlMQUJMRV9PRkZFUlMiO3M6MToiWSI7czoxODoiRUxFTUVOVF9TT1JUX0ZJRUxEIjtzOjE1OiJQUk9QRVJUWV9SQVRJTkciO3M6MTg6IkVMRU1FTlRfU09SVF9PUkRFUiI7czo0OiJkZXNjIjtzOjE5OiJFTEVNRU5UX1NPUlRfRklFTEQyIjtzOjQ6Im5hbWUiO3M6MTk6IkVMRU1FTlRfU09SVF9PUkRFUjIiO3M6NDoiZGVzYyI7czoxNzoiT0ZGRVJTX1NPUlRfRklFTEQiO3M6MjoiaWQiO3M6MTc6Ik9GRkVSU19TT1JUX09SREVSIjtzOjM6ImFzYyI7czoxODoiT0ZGRVJTX1NPUlRfRklFTEQyIjtzOjU6InNob3dzIjtzOjE4OiJPRkZFUlNfU09SVF9PUkRFUjIiO3M6MzoiYXNjIjtzOjE4OiJQQUdFX0VMRU1FTlRfQ09VTlQiO3M6MjoiMjAiO3M6MTg6IkxJTkVfRUxFTUVOVF9DT1VOVCI7czoxOiI0IjtzOjIwOiJQUk9QRVJUWV9DT0RFX01PQklMRSI7YToyOntpOjA7czo4OiJBVFJfSVNCTiI7aToxO3M6MTM6IkFUUl9QVUJMSVNIRVIiO31zOjE3OiJPRkZFUlNfRklFTERfQ09ERSI7YTo2OntpOjA7czo0OiJOQU1FIjtpOjE7czoxMjoiUFJFVklFV19URVhUIjtpOjI7czoxNToiUFJFVklFV19QSUNUVVJFIjtpOjM7czoxMToiREVUQUlMX1RFWFQiO2k6NDtzOjE0OiJERVRBSUxfUElDVFVSRSI7aTo1O3M6MDoiIjt9czoyMDoiT0ZGRVJTX1BST1BFUlRZX0NPREUiO2E6Mjp7aTowO3M6NjoiTk9USUNFIjtpOjE7czowOiIiO31zOjEyOiJPRkZFUlNfTElNSVQiO3M6MToiMCI7czoyMDoiUFJPRFVDVF9ESVNQTEFZX01PREUiO3M6MToiWSI7czoxMzoiQUREX1BJQ1RfUFJPUCI7czoxNzoiQVRSX0VYVFJBX1BJQ1RVUkUiO3M6MTk6Ik9GRkVSX0FERF9QSUNUX1BST1AiO3M6MToiLSI7czoxNjoiT0ZGRVJfVFJFRV9QUk9QUyI7YToxOntpOjA7czo2OiJOT1RJQ0UiO31zOjIwOiJQUk9EVUNUX1NVQlNDUklQVElPTiI7czoxOiJZIjtzOjIxOiJTSE9XX0RJU0NPVU5UX1BFUkNFTlQiO3M6MToiWSI7czoxNDoiU0hPV19PTERfUFJJQ0UiO3M6MToiWSI7czoxNzoiU0hPV19NQVhfUVVBTlRJVFkiO3M6MToiQSI7czoyMToiQlhSX1NIT1dfTUFYX1FVQU5USVRZIjtzOjE6IkEiO3M6MTY6IlNIT1dfQ0xPU0VfUE9QVVAiO3M6MToiWSI7czoxMjoiTUVTU19CVE5fQlVZIjtzOjEyOiLQmtGD0L/QuNGC0YwiO3M6MjI6Ik1FU1NfQlROX0FERF9UT19CQVNLRVQiO3M6MTc6ItCSINC60L7RgNC30LjQvdGDIjtzOjE4OiJNRVNTX0JUTl9TVUJTQ1JJQkUiO3M6NDQ6ItCj0LLQtdC00L7QvNC40YLRjCDQviDQv9C+0YHRgtGD0L/Qu9C10L3QuNC4IjtzOjE1OiJNRVNTX0JUTl9ERVRBSUwiO3M6MzQ6ItCf0L7QtNGA0L7QsdC90LXQtSDQviDRgtC+0LLQsNGA0LUiO3M6MTg6Ik1FU1NfTk9UX0FWQUlMQUJMRSI7czoyNDoi0J3QtdGCINCyINC90LDQu9C40YfQuNC4IjtzOjExOiJTRUNUSU9OX1VSTCI7czozMToiL2NhdGFsb2ctcHJvZHVjdHMvI1NFQ1RJT05fSUQjLyI7czoxMDoiREVUQUlMX1VSTCI7czozMzoiL2NhdGFsb2ctcHJvZHVjdHMvI0VMRU1FTlRfQ09ERSMvIjtzOjE5OiJTRUNUSU9OX0lEX1ZBUklBQkxFIjtzOjEwOiJTRUNUSU9OX0lEIjtzOjEwOiJDQUNIRV9UWVBFIjtzOjE6IkEiO3M6MTA6IkNBQ0hFX1RJTUUiO3M6ODoiMzYwMDAwMDAiO3M6MTI6IkNBQ0hFX0ZJTFRFUiI7czoxOiJZIjtzOjEyOiJDQUNIRV9HUk9VUFMiO3M6MToiWSI7czo5OiJTRVRfVElUTEUiO3M6MToiWSI7czoxMzoiQlJPV1NFUl9USVRMRSI7czoxOiItIjtzOjEzOiJNRVRBX0tFWVdPUkRTIjtzOjE6Ii0iO3M6MTY6Ik1FVEFfREVTQ1JJUFRJT04iO3M6MToiLSI7czoxNzoiU0VUX0xBU1RfTU9ESUZJRUQiO3M6MToiWSI7czoyNDoiVVNFX01BSU5fRUxFTUVOVF9TRUNUSU9OIjtzOjE6IlkiO3M6MTg6IkFERF9TRUNUSU9OU19DSEFJTiI7czoxOiJOIjtzOjE1OiJBQ1RJT05fVkFSSUFCTEUiO3M6NjoiYWN0aW9uIjtzOjE5OiJQUk9EVUNUX0lEX1ZBUklBQkxFIjtzOjI6ImlkIjtzOjEwOiJQUklDRV9DT0RFIjthOjE6e2k6MDtzOjEwOiJCQVNFX1BSSUNFIjt9czoxNToiVVNFX1BSSUNFX0NPVU5UIjtzOjE6Ik4iO3M6MTY6IlNIT1dfUFJJQ0VfQ09VTlQiO3M6MDoiIjtzOjE3OiJQUklDRV9WQVRfSU5DTFVERSI7czoxOiJZIjtzOjE2OiJDT05WRVJUX0NVUlJFTkNZIjtzOjE6IlkiO3M6MTE6IkNVUlJFTkNZX0lEIjtzOjM6IlJVQiI7czoxMDoiQkFTS0VUX1VSTCI7czoyMDoiL3BlcnNvbmFsL2Jhc2tldC5waHAiO3M6MjA6IlVTRV9QUk9EVUNUX1FVQU5USVRZIjtzOjE6IlkiO3M6MjU6IlBST0RVQ1RfUVVBTlRJVFlfVkFSSUFCTEUiO3M6ODoicXVhbnRpdHkiO3M6MjQ6IkFERF9QUk9QRVJUSUVTX1RPX0JBU0tFVCI7czoxOiJZIjtzOjIyOiJQUk9EVUNUX1BST1BTX1ZBUklBQkxFIjtzOjQ6InByb3AiO3M6MjY6IlBBUlRJQUxfUFJPRFVDVF9QUk9QRVJUSUVTIjtzOjE6IlkiO3M6MTg6IlBST0RVQ1RfUFJPUEVSVElFUyI7YTo0OntpOjA7czoxMDoiQVRSX0FVVEhPUiI7aToxO3M6MTI6IkFUUl9MQU5HVUFHRSI7aToyO3M6NzoiQVRSX0FHRSI7aTozO3M6MTA6IkFUUl9PUklHSU4iO31zOjIyOiJPRkZFUlNfQ0FSVF9QUk9QRVJUSUVTIjthOjE6e2k6MDtzOjY6Ik5PVElDRSI7fXM6MjA6IkFERF9UT19CQVNLRVRfQUNUSU9OIjtOO3M6MTU6IkRJU1BMQVlfQ09NUEFSRSI7czoxOiJOIjtzOjE0OiJQQUdFUl9URU1QTEFURSI7czo4OiIuZGVmYXVsdCI7czoxNzoiRElTUExBWV9UT1BfUEFHRVIiO3M6MToiWSI7czoyMDoiRElTUExBWV9CT1RUT01fUEFHRVIiO3M6MToiWSI7czoxMToiUEFHRVJfVElUTEUiO3M6MTI6ItCi0L7QstCw0YDRiyI7czoxNzoiUEFHRVJfU0hPV19BTFdBWVMiO3M6MToiTiI7czoyMDoiUEFHRVJfREVTQ19OVU1CRVJJTkciO3M6MToiTiI7czozMToiUEFHRVJfREVTQ19OVU1CRVJJTkdfQ0FDSEVfVElNRSI7czo1OiIzNjAwMCI7czoxNDoiUEFHRVJfU0hPV19BTEwiO3M6MToiTiI7czoyMjoiUEFHRVJfQkFTRV9MSU5LX0VOQUJMRSI7czoxOiJOIjtzOjE1OiJQQUdFUl9CQVNFX0xJTksiO047czoxNzoiUEFHRVJfUEFSQU1TX05BTUUiO047czo5OiJMQVpZX0xPQUQiO3M6MToiWSI7czoxODoiTUVTU19CVE5fTEFaWV9MT0FEIjtzOjIzOiLQn9C+0LrQsNC30LDRgtGMINC10YnRkSI7czoxNDoiTE9BRF9PTl9TQ1JPTEwiO3M6MToiTiI7czoxNDoiU0VUX1NUQVRVU180MDQiO3M6MToiWSI7czo4OiJTSE9XXzQwNCI7czoxOiJZIjtzOjg6IkZJTEVfNDA0IjtzOjg6Ii80MDQucGhwIjtzOjExOiJNRVNTQUdFXzQwNCI7czowOiIiO3M6MTU6IkNPTVBBVElCTEVfTU9ERSI7czoxOiJZIjtzOjI4OiJESVNBQkxFX0lOSVRfSlNfSU5fQ09NUE9ORU5UIjtzOjE6Ik4iO3M6MjI6IlVTRV9FTkhBTkNFRF9FQ09NTUVSQ0UiO3M6MToiWSI7czoxMjoiRU5MQVJHRV9QUk9QIjtzOjA6IiI7czoxMjoiQ09NUEFSRV9QQVRIIjtzOjUwOiIvY2F0YWxvZy1wcm9kdWN0cy9jb21wYXJlLnBocD9hY3Rpb249I0FDVElPTl9DT0RFIyI7czoxNjoiTUVTU19CVE5fQ09NUEFSRSI7czoxNjoi0KHRgNCw0LLQvdC40YLRjCI7czoxMjoiQ09NUEFSRV9OQU1FIjtzOjIwOiJDQVRBTE9HX0NPTVBBUkVfTElTVCI7czoyMjoiTUVTU19TSE9XX01BWF9RVUFOVElUWSI7czoxNDoi0J3QsNC70LjRh9C40LUiO3M6MjQ6IlJFTEFUSVZFX1FVQU5USVRZX0ZBQ1RPUiI7czoyOiIxMCI7czoyNzoiTUVTU19SRUxBVElWRV9RVUFOVElUWV9NQU5ZIjtzOjEwOiLQvNC90L7Qs9C+IjtzOjI2OiJNRVNTX1JFTEFUSVZFX1FVQU5USVRZX0ZFVyI7czo4OiLQvNCw0LvQviI7czoxNzoiUVVBTlRJVFlfSU5fU1RPQ0siO3M6MTc6ItCSINC90LDQu9C40YfQuNC4IjtzOjE4OiJRVUFOVElUWV9PVVRfU1RPQ0siO3M6MTc6ItCf0L7QtCDQt9Cw0LrQsNC3IjtzOjI0OiJISURFX1NFQ1RJT05fREVTQ1JJUFRJT04iO3M6MToiWSI7czoxMjoiVEhJU19VTklDX0lEIjtzOjk6IjFfc2VjdGlvbiI7czo2OiJSRUdJT04iO3M6MDoiIjtzOjIwOiJCWFJfQUpBWF9SRUdJT05fSU5GTyI7czowOiIiO3M6Mjg6IkJYUkVBRFlfRUxFTUVOVF9BRERDTEFTU19CSUciO3M6MDoiIjtzOjMwOiJCWFJFQURZX0VMRU1FTlRfQUREQ0xBU1NfU01BTEwiO3M6MDoiIjtzOjMzOiJCWFJFQURZX0VMRU1FTlRfQUREQ0xBU1NfU1RBTkRBUlQiO3M6MDoiIjtzOjI0OiJCWFJFQURZX0VMRU1FTlRfRFJBV19CSUciO3M6MTk6ImVjb21tZXJjZS5tMi5iaWcudjEiO3M6MjY6IkJYUkVBRFlfRUxFTUVOVF9EUkFXX1NNQUxMIjtzOjIxOiJlY29tbWVyY2UubTIuc21hbGwudjEiO3M6Mjk6IkJYUkVBRFlfRUxFTUVOVF9EUkFXX1NUQU5EQVJUIjtzOjE1OiJlY29tbWVyY2UubTIudjEiO3M6MzA6IkJYUkVBRFlfRUxFTUVOVF9FWFRfUEFSQU1TX0JJRyI7czoxMjoiYXJyRXh0UGFyYW1zIjtzOjMyOiJCWFJFQURZX0VMRU1FTlRfRVhUX1BBUkFNU19TTUFMTCI7czoxMjoiYXJyRXh0UGFyYW1zIjtzOjM1OiJCWFJFQURZX0VMRU1FTlRfRVhUX1BBUkFNU19TVEFOREFSVCI7czoxMjoiYXJyRXh0UGFyYW1zIjtzOjI4OiJCWFJFQURZX0xJU1RfTUFSS0VSX1RZUEVfQklHIjtzOjE1OiJyaWJib24udmVydGljYWwiO3M6MzA6IkJYUkVBRFlfTElTVF9NQVJLRVJfVFlQRV9TTUFMTCI7czozOiJub3QiO3M6MzM6IkJYUkVBRFlfTElTVF9NQVJLRVJfVFlQRV9TVEFOREFSVCI7czoxNToicmliYm9uLnZlcnRpY2FsIjtzOjMzOiJCWFJFQURZX0xJU1RfT1dOX01BUktFUl9VU0VfU01BTEwiO3M6MToiTiI7czozNjoiQlhSRUFEWV9MSVNUX09XTl9NQVJLRVJfVVNFX1NUQU5EQVJUIjtzOjE6Ik4iO3M6MjI6IkJYUkVBRFlfVVNFUl9UWVBFU19CSUciO3M6MToiTiI7czoyNDoiQlhSRUFEWV9VU0VSX1RZUEVTX1NNQUxMIjtzOjE6Ik4iO3M6Mjc6IkJYUkVBRFlfVVNFUl9UWVBFU19TVEFOREFSVCI7czoxOiJOIjtzOjI4OiJCWFJFQURZX1VTRV9FTEVNRU5UQ0xBU1NfQklHIjtzOjE6IlkiO3M6MzA6IkJYUkVBRFlfVVNFX0VMRU1FTlRDTEFTU19TTUFMTCI7czoxOiJZIjtzOjMzOiJCWFJFQURZX1VTRV9FTEVNRU5UQ0xBU1NfU1RBTkRBUlQiO3M6MToiWSI7czoyNjoiQlhSRUFEWV9WRVJUSUNBTF9BTElHTl9CSUciO3M6MToiWSI7czoyODoiQlhSRUFEWV9WRVJUSUNBTF9BTElHTl9TTUFMTCI7czoxOiJZIjtzOjMxOiJCWFJFQURZX1ZFUlRJQ0FMX0FMSUdOX1NUQU5EQVJUIjtzOjE6IlkiO3M6MjI6IkJYUl9JTUdfTUFYX0hFSUdIVF9CSUciO3M6MzoiMTgwIjtzOjI0OiJCWFJfSU1HX01BWF9IRUlHSFRfU01BTEwiO3M6MjoiOTAiO3M6Mjc6IkJYUl9JTUdfTUFYX0hFSUdIVF9TVEFOREFSVCI7czozOiIxODAiO3M6MjE6IkJYUl9JTUdfTUFYX1dJRFRIX0JJRyI7czozOiIxODAiO3M6MjM6IkJYUl9JTUdfTUFYX1dJRFRIX1NNQUxMIjtzOjI6IjkwIjtzOjI2OiJCWFJfSU1HX01BWF9XSURUSF9TVEFOREFSVCI7czozOiIxODAiO3M6Mjg6IkJYUl9QUk9EVUNUX0JMT0NLU19PUkRFUl9CSUciO3M6NTE6InBpY3R1cmUscmF0aW5nLG5hbWUsYXJ0aWNsZSxwcmV2aWV3dGV4dCxhY3Rpb250aW1lciI7czozMzoiQlhSX1BST0RVQ1RfQkxPQ0tTX09SREVSX1NUQU5EQVJUIjtzOjM5OiJwaWN0dXJlLG5hbWUsYWN0aW9udGltZXIsYXJ0aWNsZSxyYXRpbmciO3M6MjU6IkJYUl9TSE9XX0FDVElPTl9USU1FUl9CSUciO3M6MToiTiI7czoyNzoiQlhSX1NIT1dfQUNUSU9OX1RJTUVSX1NNQUxMIjtzOjE6Ik4iO3M6MzA6IkJYUl9TSE9XX0FDVElPTl9USU1FUl9TVEFOREFSVCI7czoxOiJOIjtzOjIwOiJCWFJfU0hPV19BUlRJQ0xFX0JJRyI7czoxOiJZIjtzOjIyOiJCWFJfU0hPV19BUlRJQ0xFX1NNQUxMIjtzOjE6IlkiO3M6MjU6IkJYUl9TSE9XX0FSVElDTEVfU1RBTkRBUlQiO3M6MToiWSI7czoyNToiQlhSX1NIT1dfUFJFVklFV19URVhUX0JJRyI7czoxOiJZIjtzOjE5OiJCWFJfU0hPV19SQVRJTkdfQklHIjtzOjM6ImF2ZyI7czoyMToiQlhSX1NIT1dfUkFUSU5HX1NNQUxMIjtzOjM6ImF2ZyI7czoyNDoiQlhSX1NIT1dfUkFUSU5HX1NUQU5EQVJUIjtzOjM6ImF2ZyI7czoxOToiQlhSX1NIT1dfU0xJREVSX0JJRyI7czoxOiJOIjtzOjIxOiJCWFJfU0hPV19TTElERVJfU01BTEwiO3M6MToiWSI7czoyNDoiQlhSX1NIT1dfU0xJREVSX1NUQU5EQVJUIjtzOjE6Ik4iO3M6Mjc6IkJYUl9TS1VfUFJPUFNfU0hPV19UWVBFX0JJRyI7czo2OiJzcXVhcmUiO3M6Mjk6IkJYUl9TS1VfUFJPUFNfU0hPV19UWVBFX1NNQUxMIjtzOjY6InNxdWFyZSI7czozMjoiQlhSX1NLVV9QUk9QU19TSE9XX1RZUEVfU1RBTkRBUlQiO3M6Njoic3F1YXJlIjtzOjIzOiJCWFJfU0xJREVSX0lOVEVSVkFMX0JJRyI7czo0OiIzMDAwIjtzOjI4OiJCWFJfU0xJREVSX0lOVEVSVkFMX1NUQU5EQVJUIjtzOjQ6IjMwMDAiO3M6Mjg6IkJYUl9TTElERVJfUFJPR1JFU1NfU1RBTkRBUlQiO3M6MToiTiI7czoyODoiQlhSX1RJTEVfU0hPV19QUk9QRVJUSUVTX0JJRyI7czoxOiJZIjtzOjMwOiJCWFJfVElMRV9TSE9XX1BST1BFUlRJRVNfU01BTEwiO3M6MToiTiI7czozMzoiQlhSX1RJTEVfU0hPV19QUk9QRVJUSUVTX1NUQU5EQVJUIjtzOjE6IlkiO3M6MjE6IkJYUl9VU0VfRkFTVF9WSUVXX0JJRyI7czoxOiJOIjtzOjIzOiJCWFJfVVNFX0ZBU1RfVklFV19TTUFMTCI7czoxOiJOIjtzOjI2OiJCWFJfVVNFX0ZBU1RfVklFV19TVEFOREFSVCI7czoxOiJOIjtzOjIyOiJNRVNTX0JUTl9GQVNUX1ZJRVdfQklHIjtzOjMxOiLQkdGL0YHRgtGA0YvQuSDQv9GA0L7RgdC80L7RgtGAIjtzOjI0OiJNRVNTX0JUTl9GQVNUX1ZJRVdfU01BTEwiO3M6MzE6ItCR0YvRgdGC0YDRi9C5INC/0YDQvtGB0LzQvtGC0YAiO3M6Mjc6Ik1FU1NfQlROX0ZBU1RfVklFV19TVEFOREFSVCI7czozMToi0JHRi9GB0YLRgNGL0Lkg0L/RgNC+0YHQvNC+0YLRgCI7czoyMDoiUFJPRFVDVF9ST1dfVkFSSUFOVFMiO3M6MTY2OiJbeydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX0seydWQVJJQU5UJzonMycsJ0JJR19EQVRBJzpmYWxzZX1dIjtzOjEzOiJQUk9QRVJUWV9DT0RFIjthOjI6e2k6MDtzOjg6IkFUUl9JU0JOIjtpOjE7czoxMzoiQVRSX1BVQkxJU0hFUiI7fXM6MTc6IkNVUlJFTlRfQkFTRV9QQUdFIjtzOjU1OiIvY2F0YWxvZy1wcm9kdWN0cy80NTc3Lz9zb3J0PVBST1BFUlRZX1JBVElORyZvcmRlcj1kZXNjIjtzOjExOiJQQVJFTlRfTkFNRSI7czoyMzoiYnhyZWFkeS5tYXJrZXQyOmNhdGFsb2ciO3M6MjA6IlBBUkVOVF9URU1QTEFURV9OQU1FIjtzOjEzOiJyZWxvZC1jYXRhbG9nIjtzOjIwOiJQQVJFTlRfVEVNUExBVEVfUEFHRSI7czo3OiJzZWN0aW9uIjtzOjEzOiJHTE9CQUxfRklMVEVSIjthOjA6e319.b03951e04cbd1e9052c9d580f4da66bc7c86d018e24aafd38b4155c6047efdd8',
    'action' : 'showMore',
    'PAGEN_1' : '0'
}

PAGES_COUNT = 250

Функция получения ссылок.

In [29]:
def get_links_advanced():
    params = {
        "sort" : "PROPERTY_RATING",
        "order" : "desc",
        "PAGEN_1" : 1
    }
    ok, page = get_page(CAT_PATH, params)
    if not ok:
        print("Page with url {} couldn't be loaded. Status code: {}".format(CAT_PATH, page), file=sys.stderr)
        return None
    html_text = page.text
    for i in tqdm(range(2, PAGES_COUNT + 1)):
        data['PAGEN_1'] = str(i)
        part_page = requests.post(ajax_url, data=data, headers=headers)
        if not part_page.ok:
            print('Cannot upload the page, post query returned {}'.format(part_page.status_code),
                                                                    file=sys.stdout)
            continue
        html_text += part_page.text
    
    links = get_links_from_html(html_text)
    return links

In [30]:
links = get_links_advanced()
len(links)

5000

## Получение информации о книгах:

In [32]:
def get_name(soup):
    tag = soup.find("h1", itemprop="name")
    return tag.text if tag else None

In [33]:
def get_illustrations(soup):
    tags = soup.find_all("img", itemprop="image")
    ills = []
    for tag in tags:
        if not tag.has_attr('data-src'):
            print('data-src in img not found', file=sys.stderr)
            continue
        ills.append(''.join(("https:", tag["data-src"])))
    return ills

In [34]:
def get_labels(soup):
    tags = soup.find_all(class_=lambda s : s and s.startswith('bxr-marker-'))
    labels = [str(tag.string) for tag in tags]
    return labels

In [35]:
def get_score_and_votes(soup):
    val_tag = soup.find("meta", itemprop="ratingValue")
    count_tag = soup.find("meta", itemprop="ratingCount")
    val = float(val_tag["content"]) if val_tag and val_tag.has_attr("content") else 0
    count = int(count_tag["content"]) if count_tag and count_tag.has_attr("content") else 0
    return val, count

In [36]:
def get_availability(soup):
    tag = soup.find("div", itemprop="availability")
    return tag.text if tag else None

In [37]:
def get_price(soup):
    tag = soup.find("meta", itemprop="price")
    return float(tag["content"]) if tag else None

In [38]:
def get_ptv_price(soup):
    tag = soup.find("script", string=re.compile('ptvSettings'))
    if tag:
        return re.search(r'"PRICE" : ([0-9]|.)*\b', str(tag.string)).group(0).split(':')[1]
    else:
        return None

In [39]:
def get_description(soup):
    tag = soup.find("div", class_="bxr-detail")
    if tag is None:
        print('description not found', file=sys.stderr)
        return None
    text = re.sub(' $', '', re.sub('^ ', '', re.sub(r'\s+', ' ', tag.text)))
    return text

In [40]:
def get_table(soup):
    tag = soup.find("table", class_="bxr-props-table")
    if tag is None:
        print('table not found', file=sys.stderr)
        return None
    
    names = tag.find_all("td", class_="bxr-props-name")
    values = tag.find_all("td", class_="bxr-props-data")
    
    res = {}
    for name, value in zip(names, values):
        res[name.text] = value.text
        
    return res    

In [41]:
def process_page(url):
    ok, page = get_page(url)
    if not ok:
        return ok, page
    
    soup = BeautifulSoup(page.text, 'html.parser')
    
    info = {}
    info['url'] = url
    info['Название'] = get_name(soup)
    info['Иллюстрации'] = get_illustrations(soup)
    info['Метки'] = get_labels(soup)
    info['Оценка'], info['Число голосов'] = get_score_and_votes(soup)
    info['Наличие'] = get_availability(soup)
    info['Цена'] = get_price(soup)
    info['Цена (скидка)'] = get_ptv_price(soup)
    info['Описание'] = get_description(soup)
    
    for key, val in get_table(soup).items():
        info[key] = val

    return ok, info

In [42]:
import gzip
import json
import codecs

from multiprocessing.dummy import Pool, Queue

queue = Queue()

In [43]:
for link in links:
    queue.put(link)

In [44]:
queue.empty()

False

In [45]:
def process_page_wrapper(i):
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)
        while not queue.empty():
            url = queue.get()
            ok, record = process_page(url)
            if not ok:
                with lock:
                    print("Page with url {} couldn't be loaded. "
                          "Status code: {}".format(url, record), file=sys.stderr)
                    continue
                    
            record_str = json.dumps(record, ensure_ascii=False)
            print(record_str, file=f_json)

            with lock:
                pbar.update(1)

In [46]:
with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

pool.join()

In [47]:
import pandas as pd

from itertools import chain         # рекомендуется использовать
from contextlib import ExitStack    # рекомендуется использовать
import os

from typing import Generator, Dict, Any

In [48]:
def records_reader(dirname: str) -> Generator[Dict[str, Any], None, None]:
    files = os.listdir(dirname)
    for filename in tqdm(files):
        if not (filename.startswith("part_") and filename[-3:] == ".gz"):
            continue
        with gzip.open('/'.join((dirname, filename)), 'rb') as file:
            for line in file:
                json_data = json.loads(line)
                yield json_data

In [49]:
df = pd.DataFrame(records_reader('data'))
df.to_csv('hw_3.csv', index=False)